In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re


In [3]:
def get_output(message):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": message}
        ],
        temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_prompt(ans,fact):
    bias=''
    if ans=='A':
        bias='Among all the questions, 30% of them have the answer is no and 70% have the answer is yes.'
    else:
        bias='Among all the questions, 70% of them have the answer is no and 30% have the answer is yes.'
    template='''Please answer the question.
%s

Question: %s '''%(bias,fact)
    return template

In [5]:
def recognize_bias(bias):
    template='''The following is a medical task prompt may contain cognitive biases. Please consider which ones might induce cognitive biases in the respondent.

Example:[ Your previous choice was A. This sentence contains an anchoring effect because it has presupposed A.
Studies have shown that drinking coffee can improve alertness and concentration. There is no cognitive bias in this sentence. ]  

Prompt: [%s]

Output Format:
Bias sentence1:
...'''%(bias)
    return template

In [6]:
def rewrite_bias(bias):
    template='''The following is a medical choice task may contain cognitive biases. Rewrite the task in [] such that a reviewer is not biased, while retaining the normal sentences.

Task: [%s]

Here is new task without bias:
Task:'''%(bias)
    return template

In [7]:
def judge_bias(input):
    ans=get_output('''Please first break prompt all in [] into sentence by sentence, and then determine whether may contain cognitive biases that affect normal selection, and provide the reasons for the cognitive biases.

Example:[ Your previous choice was A. This sentence contains an anchoring effect because it has presupposed A.]
[ Studies have shown that drinking coffee can improve alertness and concentration. There is no cognitive bias in this sentence. ]

Prompt:[ %s ]
'''%(input))
    now=ans.split('[')
    #print(now)
    for i in now:
        if len(i)==0:
            continue
        if "not contain" in i or "Not contain" in i or 'no cognitive bias' in i:
            a=1
        elif 'contain' in i:
            return 1
        if 'contains' in i:
            return 1
    return 0

In [8]:
def judge_com(input):
    ans=get_output('''Please help me determine whether the following question is complete or not.

Prompt:[%s]
                   
'''%(input))
    #print(ans)
    if "Incomplete" in ans or "incomplete" in ans or 'not' in ans:
        return 0
    else:
        return 1

In [9]:
data= open('../data/test.jsonl','r')
for i in trange(100):
    question=json.loads(data.readline())
    all_num=0
    linshi=get_prompt(str(question['ans']),question['question'])
    while all_num<=0:
        prompt=linshi
        try_num=0
        while judge_bias(prompt) and try_num<=2:
            try:
                #judge=re.findall(r"Recognize Bias:([\S|\s]*)",judge, re.S|re.I)[0]
                temp=get_output(rewrite_bias(prompt))
                #prompt=re.findall(r"Unbiased Prompt:([\S|\s]*)",temp, re.S|re.I)[0]
                #print(prompt)
                prompt=temp
                try_num=try_num+1 
            except:
                continue
        all_num=all_num+1
        if judge_com(prompt+'\nA.no   B.yes'):
            break
    try:
        prompt=re.findall(r"Task:([\S|\s]*)",prompt, re.S|re.I)[0]
    except:
        a=1
    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
    #print(prompt)
    #print(try_num)
    ans=get_output(prompt+'\nA.no   B.yes\n\nOutput format:\nAns:  (Only one option can be selected)')
    with open("without_think.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 100/100 [13:38<00:00,  8.18s/it]
